# Import OpenVisus


In [1]:
import os,sys,logging,time
import numpy as np

print(sys.executable)

os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"
os.environ["BOKEH_LOG_LEVEL"]="debug"
os.environ["VISUS_CPP_VERBOSE"]="0"
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
os.environ["VISUS_VERBOSE_DISKACCESS"]="0"

import panel as pn
pn.extension(log_level="DEBUG",notifications=True, sizing_mode="stretch_width")

if True:
    sys.path.append("c:/projects/openvisus/build/RelWithDebInfo")
    sys.path.append("c:/projects/openvisuspy/src")

from openvisuspy import Slice, SetupJupyterLogger, LoadDataset, ExecuteBoxQuery
logger=SetupJupyterLogger(logging_level=logging.DEBUG) 
print(time.time(),"OpenVisus imported")

C:\projects\openvisuspy\.venv\Scripts\python.exe


NotificationArea()

1707381951.8433998 OpenVisus imported


In [2]:
# if you have problems here see # https://github.com/holoviz/holoviews/issues/4861
# you need to (1) Restart and Clear All Cells (2) save the notebook (3) kill jupyter lab (4) restart
button = pn.widgets.Button(name="Is Panel working? Click me...")
def onClick(evt):
    button.name="Yes"
button.on_click(onClick)
app=pn.Row(button)
app

Row(sizing_mode='stretch_width')
    [0] Button(name='Is Panel working? C..., sizing_mode='stretch_width')

# Example of loading data from Object storage

In [3]:
endpoint_url="https://maritime.sealstorage.io/api/v0/s3"
url=f"{endpoint_url}/utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx?cached=idx& access_key=any&secret_key=any&endpoint_url={endpoint_url}"

db=LoadDataset(url)
W,H,D=db.getLogicBox()[1]
access=db.createAccess()

# get a Z slice in the middle to compute the range of the data
endh=db.getMaxResolution()-6
Z=D//2
logic_box, delta, num_pixels=db.getAlignedBox([[0,0,Z],[W,H,Z]], endh, slice_dir=2)
data=list(ExecuteBoxQuery(db, access=access, logic_box=logic_box,  endh=endh, num_refinements=1))[0]['data']
palette_range = np.min(data)/4, np.max(data)/4  
print("logic box",W,H,D) 
print("palette_range",palette_range)
print("resolution",db.getMaxResolution())

logic box 8640 6480 90
palette_range (-0.25256651639938354, 0.3600933849811554)
resolution 34


# Show single slice of a RGB 2D dataset (David)

In [6]:
def CreateApp():
    view=Slice()
    view.load({
     "datasets": [
         {
             "name":"david", 
             "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=david_subsampled&cached=idx"
         },
         {
             "name":"2kbit1",
             "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1&cached=idx"
         },
         {
             "name":"retina",
             "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit&cached=idx"
         },
         {
             "name":"chess-zip",
             "url":"http://atlantis.sci.utah.edu:80/mod_visus?dataset=chess-zip&cached=idx",
             "palette" :"Viridis256",
             "range-min": -0.017141795,
             "range-max": +0.012004322,
         },
         {
             "name":"chess-recon",
             "url":"http://atlantis.sci.utah.edu:80/mod_visus?dataset=chess-recon_combined_1_2_3_fullres_zip&cached=idx",
             "palette" :"Plasma256",
             "range-min": -0.0014,
             "range-max": +0.0020,   
         },
         {
            "name": "llc2160_arco",
            "url": f"{endpoint_url}/utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx?cached=idx& access_key=any&secret_key=any&endpoint_url={endpoint_url}",
            "palette":"colorcet.coolwarm",
            "range-min":-0.25256651639938354,
            "range-max":+0.3600933849811554,
            "timestep-delta":10,
            "timestep": 2015,
            "resolution": -6, 
         },
         {
            "name":"bellows",
            "url": "http://atlantis.sci.utah.edu/mod_visus?dataset=bellows_CT_NASA_JHochhalter&cached=idx",
            "palette":"Greys256",
            "range-min":0,
            "range-max":65536
         } 
         ] + [ 
            {
                "name": f"diamond-{zone}", 
                "url": f"{endpoint_url}/utah/nasa/dyamond/idx_arco/face{zone}/u_face_{zone}_depth_52_time_0_10269.idx?cached=idx& access_key=any&secret_key=any&endpoint_url={endpoint_url}",
                "palette": "Turbo256",
                "range-min":-30.0,
                "range-max":60.0,
                "timestep-delta":10,
                "resolution": -6, 
                "directions": {'Long':0, 'Lat':1, 'Depth':2},
                "logic-to-physic":[(0.0,1.0), (0.0,1.0), (0.0,10.0)],               
            }
            for zone in range(6)
        ] 
    })
    
    return pn.Column(view.getMainLayout(),sizing_mode="stretch_width",height=800)

app=CreateApp()
app.servable()

Column(height=800, sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_both')
        [0] Column(sizing_mode='stretch_both')
            [0] Row(sizing_mode='stretch_width')
                [0] Column(width=120)
                    [0] Spacer(height=18, sizing_mode='stretch_width')
                    [1] Row(sizing_mode='stretch_width')
                        [0] MenuButton(button_type='primary', items=[['Load/Save', ...], name='File', sizing_mode='stretch_width')
                        [1] TextInput(sizing_mode='stretch_width', visible=False)
                [1] Select(name='ViewMode', options=['1', '2', 'probe', ...], value='1', width=80)
                [2] Select(name='Scene', options=['david', '2kbit1', ...], value='david', width=180)
                [3] EditableFloatSlider(end=3, format=NumeralTickFormatter(id='3..., name='Time', sizing_mode='stretch_width', step=1)
                [4] Select(name='Speed', options=['1x', '2x', '4x', ...], value='1x', width=60)
                [5] Select(name='Field', options=['data'], value='data', width=80)
                [6] ColorMap(name='Palette', ncols=5, options={'Blues256': ('#08306b', ...}, value=('#440154', '#440255', ..., value_name='Viridis256', width=220)
                [7] Select(name='Mapper', options=['linear', 'log'], value='linear', width=80)
                [8] IntSlider(end=32, name='Res', sizing_mode='stretch_width', start=20, value=26)
                [9] Select(name='ViewDep', options={'Yes': True, ...}, value=True, width=80)
                [10] IntSlider(end=4, name='#Ref', value=2, width=80)
                [11] TextInput(sizing_mode='stretch_width', visible=False)
            [1] GridBox(ncols=1, sizing_mode='stretch_both')
                [0] Row(sizing_mode='stretch_both')
                    [0] Column(sizing_mode='stretch_both')
                        [0] Row(sizing_mode='stretch_width')
                            [0] Select(name='Direction', options={'X': 0, 'Y': 1, 'Z': 2}, value=2, width=80)
                            [1] EditableFloatSlider(end=0, format=NumeralTickFormatter(id='4..., name='Offset', sizing_mode='stretch_width', step=1)
                            [2] Select(name='Range', options=['metadata', 'user', ...], value='dynamic-acc', width=120)
                            [3] FloatInput(disabled=True, name='Min', width=80)
                            [4] FloatInput(disabled=True, name='Max', width=80)
                        [1] Row(sizing_mode='stretch_both')
                            [0] Row(sizing_mode='stretch_both')
                                [0] Bokeh(figure, sizing_mode='stretch_width')
                        [2] Row(sizing_mode='stretch_width')
                            [0] TextInput(sizing_mode='stretch_width')
                            [1] TextInput(sizing_mode='stretch_width')
                    [1] Column(sizing_mode='stretch_both', visible=False)
                        [0] Row(sizing_mode='stretch_width')
                            [0] EditableFloatSlider(format=NumeralTickFormatter(id='e..., name='X coordinate', step=1.0, width=160)
                            [1] EditableFloatSlider(format=NumeralTickFormatter(id='c..., name='Y coordinate', step=1, width=160)
                            [2] EditableRangeSlider(format=NumeralTickFormatter(id='e..., name='Range', width=250)
                            [3] RadioButtonGroup(options=['avg', 'mM', 'med', '*'], sizing_mode='stretch_width', value='avg')
                            [4] IntSlider(end=99, name='Res', start=20, value=24, width=100)
                            [5] IntSlider(end=8, name='#x', start=1, value=2, width=80)
                            [6] IntSlider(end=8, name='#y', start=1, value=2, width=80)
                        [1] Row(sizing_mode='stretch_width')
                            [0] Button(name='lime', sizing_mode='stretch_width')
                            [1] Button(name='red', sizing_mode='stretch_width')
                  